## Importing libraries

In [1]:
import numpy as np
import pandas as pd

## Reading Excel file

In [2]:
df = pd.read_excel('./Data_file(1).xlsx')
df.head()

,S.No.,College,Role,City type,Previous CTC,Previous job changes,Graduation marks,Exp (Months),CTC
0,1,Tier 1,Manager,Non-Metro,55523,3,66,19,71406.576531
1,2,Tier 2,Executive,Metro,57081,1,84,18,68005.870631
2,3,Tier 2,Executive,Metro,60347,2,52,28,76764.020277
3,4,Tier 3,Executive,Metro,49010,2,81,33,82092.386880
4,5,Tier 3,Executive,Metro,57879,4,74,32,73878.097729


## Finding Missing Values

In [3]:
df.nunique()

S.No.                   1338
College                    3
Role                       2
City type                  2
Previous CTC            1308
Previous job changes       4
Graduation marks          51
Exp (Months)              47
CTC                     1338
dtype: int64

In [4]:
df.shape

(1338, 9)

In [5]:
df.isnull().sum()

S.No.                   0
College                 0
Role                    0
City type               0
Previous CTC            0
Previous job changes    0
Graduation marks        0
Exp (Months)            0
CTC                     0
dtype: int64

## Pre-processing

In [6]:
clg_dummies = pd.get_dummies(df['College'])
clg_dummies.drop('Tier 3', axis=1, inplace=True)
clg_dummies.head()

,Tier 1,Tier 2
0,1,0
1,0,1
2,0,1
3,0,0
4,0,0


In [7]:
role_dummies = pd.get_dummies(df['Role'])
role_dummies.drop('Executive', axis=1, inplace=True)
role_dummies.head()

,Manager
0,1
1,0
2,0
3,0
4,0


In [8]:
city_dummies = pd.get_dummies(df['City type'])
city_dummies.drop('Non-Metro', axis=1, inplace=True)
city_dummies.head()

,Metro
0,0
1,1
2,1
3,1
4,1


In [9]:
df.drop(['S.No.','College', 'Role', 'City type'], axis=1, inplace=True)

In [10]:
new_df = pd.concat([clg_dummies, role_dummies, city_dummies,df], axis=1)
new_df.head()

,Tier 1,Tier 2,Manager,Metro,Previous CTC,Previous job changes,Graduation marks,Exp (Months),CTC
0,1,0,1,0,55523,3,66,19,71406.576531
1,0,1,0,1,57081,1,84,18,68005.870631
2,0,1,0,1,60347,2,52,28,76764.020277
3,0,0,0,1,49010,2,81,33,82092.386880
4,0,0,0,1,57879,4,74,32,73878.097729


In [11]:
new_df.drop('CTC', axis=1, inplace=True)
new_df.head()

,Tier 1,Tier 2,Manager,Metro,Previous CTC,Previous job changes,Graduation marks,Exp (Months)
0,1,0,1,0,55523,3,66,19
1,0,1,0,1,57081,1,84,18
2,0,1,0,1,60347,2,52,28
3,0,0,0,1,49010,2,81,33
4,0,0,0,1,57879,4,74,32


## Normalizing Data

In [12]:
from sklearn import preprocessing

val = new_df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
val_scaled = min_max_scaler.fit_transform(val)
normalize_df = pd.DataFrame(val_scaled)
normalize_df.columns = ['College_T1', 'College_T2', 'Role_Manager', 'City_Metro', 
                        'previous CTC', 'previous job changes', 'Graduation marks', 'Exp']
normalize_df.head()

,College_T1,College_T2,Role_Manager,City_Metro,previous CTC,previous job changes,Graduation marks,Exp
0,1.0,0.0,1.0,0.0,0.452897,0.666667,0.62,0.021739
1,0.0,1.0,0.0,1.0,0.490970,0.000000,0.98,0.000000
2,0.0,1.0,0.0,1.0,0.570783,0.333333,0.34,0.217391
3,0.0,0.0,0.0,1.0,0.293737,0.333333,0.92,0.326087
4,0.0,0.0,0.0,1.0,0.510471,1.000000,0.78,0.304348


In [13]:
df.rename(columns = {'CTC':'Actual CTC'}, inplace = True)

## Model Training

In [14]:
X = normalize_df
y = df['Actual CTC']

In [15]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

In [16]:
model.fit(X, y)

LinearRegression()

## Testing

In [17]:
test_df = pd.read_excel('./Test_data_file.xlsx')
test_df.head()

,College,Role,City type,College_T1,College_T2,Role_Manager,City_Metro,previous CTC,previous job changes,Graduation marks,Exp,Actual CTC,Predicted CTC
0,Tier 1,Manager,Non-Metro,1,0,1,0,55523,3,66,19,71406.576531,NaN
1,Tier 2,Executive,Metro,0,1,0,1,57081,1,84,18,68005.870631,NaN
2,Tier 2,Executive,Metro,0,1,0,1,60347,2,52,28,76764.020277,NaN
3,Tier 3,Executive,Metro,0,0,0,1,49010,2,81,33,82092.386880,NaN
4,Tier 3,Executive,Metro,0,0,0,1,57879,4,74,32,73878.097729,NaN


In [18]:
test_df = test_df.drop(['College', 'Role', 'City type', 'Predicted CTC'], axis='columns')
test_df.head()

,College_T1,College_T2,Role_Manager,City_Metro,previous CTC,previous job changes,Graduation marks,Exp,Actual CTC
0,1,0,1,0,55523,3,66,19,71406.576531
1,0,1,0,1,57081,1,84,18,68005.870631
2,0,1,0,1,60347,2,52,28,76764.020277
3,0,0,0,1,49010,2,81,33,82092.386880
4,0,0,0,1,57879,4,74,32,73878.097729


In [19]:
new_test_df = test_df.copy(deep=True)
new_test_df.drop('Actual CTC', axis=1, inplace=True)
new_test_df.head()

,College_T1,College_T2,Role_Manager,City_Metro,previous CTC,previous job changes,Graduation marks,Exp
0,1,0,1,0,55523,3,66,19
1,0,1,0,1,57081,1,84,18
2,0,1,0,1,60347,2,52,28
3,0,0,0,1,49010,2,81,33
4,0,0,0,1,57879,4,74,32


## Normalizing Test Data

In [21]:
test_val = new_test_df.values #returns a numpy array
test_val_scaled = min_max_scaler.fit_transform(test_val)
test_normalize_df = pd.DataFrame(test_val_scaled)
test_normalize_df.columns = ['College_T1', 'College_T2', 'Role_Manager', 'City_Metro', 
                        'previous CTC', 'previous job changes', 'Graduation marks', 'Exp']
test_normalize_df.head()

,College_T1,College_T2,Role_Manager,City_Metro,previous CTC,previous job changes,Graduation marks,Exp
0,1.0,0.0,1.0,0.0,0.452897,0.666667,0.62,0.021739
1,0.0,1.0,0.0,1.0,0.490970,0.000000,0.98,0.000000
2,0.0,1.0,0.0,1.0,0.570783,0.333333,0.34,0.217391
3,0.0,0.0,0.0,1.0,0.293737,0.333333,0.92,0.326087
4,0.0,0.0,0.0,1.0,0.510471,1.000000,0.78,0.304348


## Evaluation of Model

In [22]:
X_test = test_normalize_df
y_test = test_df['Actual CTC']

In [29]:
pred = model.predict(X_test)
prediction = pred.reshape(-1,1)
print(prediction)

[[86123.93814045]
 [65789.54652723]
 [69941.69030152]
 ...
 [64046.07444783]
 [66223.22234282]
 [90794.94065706]]


In [31]:
test_df['Actual CTC']

0       71406.576531
1       68005.870631
2       76764.020277
3       82092.386880
4       73878.097729
            ...     
1333    69712.403655
1334    69298.750099
1335    66397.770686
1336    64044.382949
1337    83346.060961
Name: Actual CTC, Length: 1338, dtype: float64

In [25]:
#Calculate root mean squared error to evaluate model performance
from sklearn.metrics import mean_squared_error
print('MSE : ', mean_squared_error(y_test,pred))

MSE :  61677937.045309596
